In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

In [3]:
import time
import csv
import numpy as np
import pandas as pd
import requests
import re
import bs4
from bs4 import BeautifulSoup as bs
from IPython.core.display import HTML

# create webdriver instance

In [21]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# automate login 

In [22]:
user = "mathis.brennan@gmail.com"
pw = "Shitballs1!"

In [7]:
def link_log(email, password):
    driver.get("https://www.linkedin.com/")
    time.sleep(3)
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()


In [23]:
link_log(user, pw)

# automate search

In [17]:
# linkedin url /pathName
urls = []
# name of connection
names = []
#primary details
p_details = []
#secondary details
s_details = []

In [9]:
s_term = 'machine learning'
s_type = 'people'

In [19]:
def li_search(search_term, search_type):
    driver.find_element_by_id("global-nav-search").click()
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    driver.implicitly_wait(10)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    button = driver.find_element_by_link_text(f"See all {search_type} results")
    driver.execute_script('arguments[0].click();',button)
    driver.implicitly_wait(5)
    page_number = 1
    current_page_url = driver.current_url
    print("current url is: " + current_page_url)
    while page_number < 10:
        print("Processing page: " + str(page_number))
        links = driver.find_elements_by_css_selector("span.entity-result__title a.app-aware-link")
        for link in links:
            name = str(link.get_attribute("innerText")).split('\n')
            names.append(name[0])
            profile_path = (str(link.get_attribute("pathname")))
            urls.append(profile_path)
        primary_deets = driver.find_elements_by_css_selector("div.entity-result__primary-subtitle.t-14.t-black")
        secondary_deets = driver.find_elements_by_css_selector('div.entity-result__secondary-subtitle.t-14')
        for x in primary_deets:
            desc = x.get_attribute("innerText")
            p_details.append(desc)
        for x in secondary_deets:
            loc = x.get_attribute("innerText")
            s_details.append(loc)
        page_number+=1
        next_page_url = current_page_url + "&page=" + str(page_number)
        print("attempting to navigate to: " + next_page_url)
        driver.get(next_page_url) 
        driver.implicitly_wait(5)
    driver.quit()

In [20]:
li_search(s_term, s_type)

current url is: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER
Processing page: 1
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER&page=2
Processing page: 2
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER&page=3
Processing page: 3
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER&page=4
Processing page: 4
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER&page=5
Processing page: 5
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=machine%20learningmachine%20learning&origin=GLOBAL_SEARCH_HEADER&page=6
Proce

# add lists to dataframe

In [898]:
name_link_df = pd.DataFrame()
name_link_df['name'] = names
name_link_df['url'] = urls
name_link_df['description'] = p_details
name_link_df['location'] = s_details
name_link_df

,name,url,description,location
0,Avery Smith,/in/averyjsmith,Data Scientist @ ExxonMobil | 👨‍💻,Greater Houston
1,Rohan Singh Rajput,/in/rohan3,Senior Data Scientist at Headspace Inc.,"Los Angeles, CA"
2,"Thom Ives, Ph.D.",/in/thomives,Lead Data Scientist at UL Prospector | Owner o...,Boise Metropolitan Area
3,Michelle Lu,/in/lu-michellepy,Data Scientist at Google,United States
4,Jalal Kiani,/in/jalal-kiani-045721122,Data Scientist,"Memphis, TN"
...,...,...,...,...
985,Sanaz Fesharaki,/in/sanazfesharaki,Economist and Data Scientist,"Columbia, MD"
986,Miao Zheng,/in/miao-zheng-0314,Data Scientist at Accenture,"Dunn Loring, VA"
987,Hardy Xu,/in/hardy-xu-03650a16b,Senior Data Scientist at BLEND360,Washington DC-Baltimore Area
988,Yayoi Furuhata,/in/yayoi-furuhata,Lead Data Scientist at Booz Allen Hamilton,"Washington, DC"


In [941]:
# create column that contains complete url for driver use
for row in name_link_df:
    name_link_df['fetch'] = 'https://www.linkedin.com' + name_link_df.url + '/'

In [25]:
fetchList = name_link_df['fetch']

In [26]:
fetchList

0                https://www.linkedin.com/in/averyjsmith
1                     https://www.linkedin.com/in/rohan3
2                   https://www.linkedin.com/in/thomives
3              https://www.linkedin.com/in/lu-michellepy
4      https://www.linkedin.com/in/jalal-kiani-045721122
                             ...                        
985           https://www.linkedin.com/in/sanazfesharaki
986          https://www.linkedin.com/in/miao-zheng-0314
987       https://www.linkedin.com/in/hardy-xu-03650a16b
988           https://www.linkedin.com/in/yayoi-furuhata
989          https://www.linkedin.com/in/melissa-dureiko
Name: fetch, Length: 990, dtype: object

# save csv

In [942]:
name_link_df.to_csv('ml_connections.csv')